# Predicting the probability of a UK Parliamentary constituency voting for the Conservative Party

## Environment Setup

In [8]:
from sklearn.linear_model import LogisticRegression as logreg
from sklearn.model_selection import train_test_split
import pandas as pd

dataset = pd.read_csv("combined-dataset-england-only.csv")


## Balancing the dataset

In order to get a better model we want there to be an equal number of consituencies which voted Tory as those who didn't

In [9]:
# Counting num of constituencies that voted Tory and for another party
total = len(dataset)
num_CON = dataset["Binary_Con (1 = win)"].sum()
num_notCON = total - num_CON

# Adjusting dataset
dataset_voteCON = dataset.loc[dataset["Binary_Con (1 = win)"] == 1].sample(num_notCON)
dataset_votenotCON = dataset.loc[dataset["Binary_Con (1 = win)"] == 0]

resampled_dataset = pd.concat((dataset_voteCON, dataset_votenotCON))

## Split the dataset into training, validation, and testing group

In order to prevent overfitting the model must be validated and tested on data which it is not trained on. Hence we randomly split the data into 3 subgroups.

In [10]:
# Put 80% of the data into a training set and 20% into a combined testing set
resampled_dataset_train, resampled_dataset_splitdata = train_test_split(
    resampled_dataset, test_size = 0.2, random_state = 0
)

# Split the combined testing set into validation and testing sets
resampled_dataset_val, resampled_dataset_test = train_test_split(
    resampled_dataset_splitdata, test_size = 0.5, random_state = 0
)